# Analisi degli interventi a sostegno dell'emergenza da COVID-19 della Regione Autonoma della Sardegna<br>
Saranno analizzati i dati relativi agli interventi a sostegno dell'emergeenza COVID-19 erogati attraverso il Sistema Informativo del Lavoro (SIL Sardegna) nel perodo che va dal Dicembre 2019 a Giugno 2021.<br>
Si tratta di due Dataset:<br>

*   SIL Sardegna_Anagrafica Avvisi.csv
*   SIL Sardegna_Elenco Domande.csv

Contenti rispettivamente l'anagrafica di tutti gli avvisi pubblici estratti dal SIL Sardegna e l'elenco di tutte le domande di partecipazione sottoposte dai potenziali beneficiari attraverso lo  stesso sistema.<br>

Nello specifico l'attività condotta consiste in:
1. Installazione dell'ambiente PySpark
2. Caricamento e prima esplorazione dei Dataset
3. Preparazione dei valori e data cleaning
3. Rimozione/Selezione delle features 
4. Creazione di nuove feature e nuovi aggregati
5. Reshaping del Dataframe 
6. Visualizzazione interattiva con Plotly e Plotly Express


![](https://databricks.com/wp-content/uploads/2018/12/PySpark-1024x164.png)


<center> 

![](http://www.regione.sardegna.it/immagini/1_240_20120705162029.gif)

</center>

##Installazione dell'ambiente


###Aggiornamento Plotly

In [1]:
!pip install plotly --upgrade

     |████████████████████████████████| 20.6MB 1.6MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


### Installazione del modulo PySpark

In [9]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz    
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [12]:
import findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

findspark.init()

In [13]:
import pyspark as ps
#from pyspark import SparkContext
sc = ps.SparkContext("local[*]",appName="RAS-Lavoro")
sc.version

ValueError: ignored

In [14]:
sc

<SparkContext master=local[*] appName=RAS-Lavoro>

###Preparazione della sessione Spark

In [15]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.appName("RAS-Lavoro") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()
spark

In [16]:
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1626338294268'),
 ('spark.app.startTime', '1626338293071'),
 ('spark.app.name', 'RAS-Lavoro'),
 ('spark.driver.port', '46817'),
 ('spark.driver.host', '06219a6dc3ea'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

###Caricamento dei package principali


In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

#imposto il template di default di Plotly
pio.templates.default = "seaborn"

#fnzione di utilità per la formattazione degli importi in €
def currency_converter(cur) :
    currency = "€ {:,.2f}".format(np.round(cur, decimals=2))
    thousands_separator = "."
    fractional_separator = ","

    if thousands_separator == ".":
        main_currency, fractional_currency = currency.split(".")[0], currency.split(".")[1]
        new_main_currency = main_currency.replace(",", ".")
        currency = new_main_currency + fractional_separator + fractional_currency
    return currency 

##Caricamento Dataset Anagrafica Avvisi


In [18]:
from pyspark.sql.types import StructField,IntegerType, StructType,StringType, DateType
struttura=[StructField('Codice Avviso',StringType(),True),
           StructField('Descrizione estesa',StringType(),True),
           StructField('Tipologia Avviso',StringType(),True),
           StructField('Assessorato / Struttura',StringType(),True),
           StructField('Servizio',StringType(),True),
           StructField('Tipo destinatario (Economico)',StringType(),True),
           StructField('Macro Categoria destinatario (Economico)',StringType(),True),
           StructField('Destinatario finale (Fruitore servizi finanziati)',StringType(),True),
           StructField('Data Apertura',DateType(),True),
           StructField('Data Chiusura',DateType(),True),
           StructField('N. domande',IntegerType(),True),
           StructField('Dotazione finanziaria',IntegerType(),True),
           StructField('Dotazione (FSE)',IntegerType(),True),
           StructField('Dotazione (Nazionale)',IntegerType(),True),
           StructField('Dotazione (Regionale)',IntegerType(),True),
           StructField('Budget prenotato',IntegerType(),True),
           StructField('Domande ammissibili',IntegerType(),True),
           StructField('Budget ammesso',IntegerType(),True),
           StructField('Budget concesso',IntegerType(),True),
           StructField('Istruttoria extra-sistema',StringType(),True)
       ]
finalStruct=StructType(fields=struttura)
df_avvisi = spark.read.csv("SIL Sardegna_Anagrafica Avvisi.csv", sep=";", header=True,
                     schema=finalStruct)

In [19]:
df_avvisi.printSchema()

root
 |-- Codice Avviso: string (nullable = true)
 |-- Descrizione estesa: string (nullable = true)
 |-- Tipologia Avviso: string (nullable = true)
 |-- Assessorato / Struttura: string (nullable = true)
 |-- Servizio: string (nullable = true)
 |-- Tipo destinatario (Economico): string (nullable = true)
 |-- Macro Categoria destinatario (Economico): string (nullable = true)
 |-- Destinatario finale (Fruitore servizi finanziati): string (nullable = true)
 |-- Data Apertura: date (nullable = true)
 |-- Data Chiusura: date (nullable = true)
 |-- N. domande: integer (nullable = true)
 |-- Dotazione finanziaria: integer (nullable = true)
 |-- Dotazione (FSE): integer (nullable = true)
 |-- Dotazione (Nazionale): integer (nullable = true)
 |-- Dotazione (Regionale): integer (nullable = true)
 |-- Budget prenotato: integer (nullable = true)
 |-- Domande ammissibili: integer (nullable = true)
 |-- Budget ammesso: integer (nullable = true)
 |-- Budget concesso: integer (nullable = true)
 |-- Ist

In [20]:
print("Dataframe shape:\nRows: ",df_avvisi.count(), " Cols: ",len(df_avvisi.columns))

Dataframe shape:
Rows:  33  Cols:  20


###Data preparation
Il Dataset "Avvisi" contiene le anagrafiche di 33 avvisi che sono stati gestiti attraverso la piattaforma SIL Sardegna a partire dal Dicembre 2019 al 24 Giugno 2021. L'estrazione dal sistema è sufficientemente ordinata e "pulita" ma occorre perfezionare alcune features.

In [21]:
#elimina spazi bianchi nella colonna "Istruttoria ..."
df_avvisi = df_avvisi.withColumn("Istruttoria extra-sistema", trim(col("Istruttoria extra-sistema")))
df_avvisi.na.fill(value=0).show()


+--------------------+--------------------+--------------------+-----------------------+--------------------+-----------------------------+----------------------------------------+-------------------------------------------------+-------------+-------------+----------+---------------------+---------------+---------------------+---------------------+----------------+-------------------+--------------+---------------+-------------------------+
|       Codice Avviso|  Descrizione estesa|    Tipologia Avviso|Assessorato / Struttura|            Servizio|Tipo destinatario (Economico)|Macro Categoria destinatario (Economico)|Destinatario finale (Fruitore servizi finanziati)|Data Apertura|Data Chiusura|N. domande|Dotazione finanziaria|Dotazione (FSE)|Dotazione (Nazionale)|Dotazione (Regionale)|Budget prenotato|Domande ammissibili|Budget ammesso|Budget concesso|Istruttoria extra-sistema|
+--------------------+--------------------+--------------------+-----------------------+--------------------

#### Formattazione della feature "Struttura di appartenenza"
Si rende necessario abbreviare la denominazione delle "Strutture proponenti" (cioè le struttore organizzative regionali che gestiscono gli avvisi) al fine di consentire una visualizzazione più agevole.

In [22]:
from pyspark.sql.functions import udf

def replace_names(string1):
    if (string1 == "Assessorato del lavoro, formazione professionale, cooperazione e sicurezza sociale"):
      return "Ass. to al Lavoro"
    elif (string1 == "ASPAL"):
      return "Agenzia del Lavoro"
    elif (string1 == "ASSESSORATO DELLA PUBBLICA ISTRUZIONE, BENI CULTURALI, INFORMAZIONE, SPETTACOLO E SPORT"):
      return "Ass. to all'Istruzione"
    else :
      return "Ass. to alla Sanità"


replace_names_udf = udf(replace_names, StringType())

df_avvisi = df_avvisi.withColumn("Assessorato / Struttura", replace_names_udf(df_avvisi["Assessorato / Struttura"]))


In [23]:
df_avvisi.select("Codice Avviso").show(35,False)

+-----------------------------------------------------+
|Codice Avviso                                        |
+-----------------------------------------------------+
|Avviso PRO.DI.GI                                     |
|Cantieri LavoRAS                                     |
|TVB Bonus under 35 - Anno 2020                       |
|TVB Bonus under 35 - Anno 2021                       |
|TVB Bonus over 35 - Anno 2020                        |
|TVB Bonus over 35 - Anno 2021                        |
|GG-2B-IeFP-2020                                      |
|Fondo di rotazione                                   |
|DSAL 2020 - Under 35                                 |
|DSAL 2020 - Over 35                                  |
|CIG Deroga 2020 covid 19                             |
|SSE Lavoro Sardegna LR 22 ART 12 comma 1 e 2         |
|SSE Lavoro Sardegna LR 22 ART 12 comma 3             |
|LR22 ART. 11 - Percorsi formativi disoccupati        |
|Avviso pubblico L.R. 5/57 Cooperative - 2020   

####Individuazione degli interventi COVID
Sono indivisuati gli interventi (Avvisi) "speciali" specificatamente pensati per fronteggiare l'emergenza da COVID 19, vengono pertanto esclusi gli Avvisi "ordinari"


In [24]:
def interventi_covid(string2):
  if (string2.startswith("Avviso")):
    return 0;
  elif (string2 =="Cantieri LavoRAS"):
    return 0;
  elif (string2.startswith("TVB")):
    return 0;
  elif (string2 =="Percorsi IeFP-2020"):
    return 0;
  elif (string2 =="(SI TORNA) TUTTI A ISCOLA - Anno Scolastico 2020/2021"):
    return 0;
  elif (string2 =="GG-2B-IeFP-2020"):
    return 0;
  elif (string2 =="Percorsi Formativi Asse 1 ed Asse 1 BIS"):
    return 0;
  elif (string2 =="IV Anno IeFP - aa.ff. 2021-2022"):
    return 0;
  elif (string2 =="SINELIMES"):
    return 0;
  else :
    return 1;

interventi_covid_udf = udf(interventi_covid, IntegerType())

df_avvisi = df_avvisi.withColumn("Intervento COVID", interventi_covid_udf(df_avvisi["Codice Avviso"]))

In [25]:
df_avvisi.select('Codice Avviso', 'Intervento COVID').show(35)

+--------------------+----------------+
|       Codice Avviso|Intervento COVID|
+--------------------+----------------+
|    Avviso PRO.DI.GI|               0|
|    Cantieri LavoRAS|               0|
|TVB Bonus under 3...|               0|
|TVB Bonus under 3...|               0|
|TVB Bonus over 35...|               0|
|TVB Bonus over 35...|               0|
|     GG-2B-IeFP-2020|               0|
|  Fondo di rotazione|               1|
|DSAL 2020 - Under 35|               1|
| DSAL 2020 - Over 35|               1|
|CIG Deroga 2020 c...|               1|
|SSE Lavoro Sardeg...|               1|
|SSE Lavoro Sardeg...|               1|
|LR22 ART. 11 - Pe...|               1|
|Avviso pubblico L...|               0|
|Avviso pubblico L...|               0|
|SSE Lavoro Sardeg...|               1|
|  Percorsi IeFP-2020|               0|
|(SI TORNA) TUTTI ...|               0|
|LR22 ART. 11 - Co...|               1|
|         LR30 ART. 5|               1|
|(R)ESISTO - Linea...|               1|


### Prima esplorazione dei dati
La prima esplorazione riguarda un confronto tra il numero di destinatari e le somme di finanziamento messe a disposizione per l'emergenza COVID nel periodo osservato e raffrontati con la totalità degli interventi.


In [26]:
# Tutti gli avvisi trovati
Stat_all_avvisi = df_avvisi.na.fill(value=0).withColumnRenamed("N. Domande","Domande").groupBy("Assessorato / Struttura")\
    .agg({'Dotazione Finanziaria':'sum',  'Domande':'sum', 'Budget Prenotato':'sum', 'Domande Ammissibili':'sum'})
# Solo avvisi classificati "Intervento COVID"
Stat_avvisi_covid = df_avvisi.filter(df_avvisi['Intervento COVID'] == 1).na.fill(value=0).withColumnRenamed("N. Domande","Domande")\
    .groupBy("Assessorato / Struttura").agg({'Dotazione Finanziaria':'sum',  'Domande':'sum', 'Budget Prenotato':'sum', 'Domande Ammissibili':'sum'})


In [27]:
Stat_avvisi_covid_pd =Stat_avvisi_covid.toPandas()
Stat_avvisi_covid_pd.fillna(0).head()

,Assessorato / Struttura,sum(Domande Ammissibili),sum(Dotazione Finanziaria),sum(Budget Prenotato),sum(Domande)
0,Agenzia del Lavoro,5131,22650000,13845948,11438
1,Ass. to al Lavoro,19365,180689000,409306735,44226


In [28]:
Stat_all_avvisi_pd =Stat_all_avvisi.toPandas()
Stat_all_avvisi_pd.fillna(0).head()


,Assessorato / Struttura,sum(Domande Ammissibili),sum(Dotazione Finanziaria),sum(Budget Prenotato),sum(Domande)
0,Agenzia del Lavoro,5131,59650000,49340324,11867
1,Ass. to al Lavoro,20082,223522411,433467229,46080
2,Ass. to all'Istruzione,0,9101000,13155971,393
3,Ass. to alla Sanità,511,3000000,3997200,681


In [29]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=Stat_all_avvisi_pd["Assessorato / Struttura"],
    x=Stat_all_avvisi_pd["sum(Dotazione Finanziaria)"],
    name='Totale interventi',
    orientation='h',
    hovertemplate = '<b><i>%{y}</i></b><br>' +
    'Totale interventi'+
    '<br><i>Dotazione</i>: <br><b>€%{x:.2f}</b><extra></extra>'
))
fig.add_trace(go.Bar(
    y=Stat_avvisi_covid_pd["Assessorato / Struttura"],
    x=Stat_avvisi_covid_pd["sum(Dotazione Finanziaria)"],
    name='Interventi Covid',
    orientation='h',
    hovertemplate = '<br><b><i>%{y}</i></b><br>'+
    'Interventi COVID'+
    '<br><i>Dotazione</i>: <br><b>€%{x:.2f}</b><extra></extra>'
))

fig.update_xaxes(title_text="Somme (€)", type="linear")
fig.update_yaxes(title_text="Struttura proponente")
fig.update_layout(title="Confronto dotazione finanziaria")
fig.update_layout(barmode='overlay', 
                  #hovermode="y", 
                  hovermode = "closest",
                  hoverlabel= { 'bgcolor': "#FFF" })
fig.show()

In [30]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=Stat_all_avvisi_pd["Assessorato / Struttura"],
    x=Stat_all_avvisi_pd["sum(Domande)"],
    name='Totale interventi',
    orientation='h',
    hovertemplate = '<br><b><i>%{y}</i></b><br>'+
    'Totale interventi'+
    '<br><i>N. Domande</i>: <br><b>%{x:i}</b><extra></extra>'
))
fig.add_trace(go.Bar(
    y=Stat_avvisi_covid_pd["Assessorato / Struttura"],
    x=Stat_avvisi_covid_pd["sum(Domande)"],
    name='Interventi Covid',
    orientation='h',
    hovertemplate = '<br><b><i>%{y}</i></b><br>'+
    'Interventi COVID'+
    '<br><i>N. Domande</i>: <br><b>%{x:i}</b><extra></extra>'
))

fig.update_xaxes(title_text="N. di partecipanti")
fig.update_yaxes(title_text="Struttura proponente")
fig.update_layout(title="Confronto partecipanti")
fig.update_layout(barmode='overlay', 
                  #hovermode="x", 
                  hovermode = "closest",
                  hoverlabel= { 'bgcolor': "#FFF" }
)
fig.show()

## Caricamento Dataset Elenco delle Domande
Viene caricato il dataset relativo alla totalità delle domande presentate dai potenziali beneficiari attraverso la piattaforma SIL Sardegna nel periodo osservato.

In [31]:
from pyspark.sql.types import StructField,IntegerType, StructType,StringType, DateType, TimestampType, FloatType, DoubleType, DecimalType
strutturaD=[StructField('Codice avviso',StringType(),True),
           StructField('Codice domanda',StringType(),True),
           StructField('Tipo destinatario',StringType(),True),
           StructField('Macrotipologia destinatario',StringType(),True),
           StructField('Sesso',StringType(),True),
           StructField('Forma giuridica',StringType(),True),
           StructField('Macro Settore ATECO',StringType(),True),
           StructField('Codice Settore ATECO',StringType(),True),
           StructField('Settore ATECO',StringType(),True),
           StructField('Comune residenza / Comune sede legale',StringType(),True),
           StructField('Provincia residenza / Provincia sede legale',StringType(),True),
           StructField('Comune domicilio / Comune sede operativa',StringType(),True),
           StructField('Provincia domicilio / Provincia sede operativa',StringType(),True),
           StructField('Data Invio',TimestampType(),True),
           StructField('Importo richiesto',StringType(),True),
           StructField('Data fine istruttoria',DateType(),True),
           StructField('ID Istruttore',StringType(),True),
           StructField('Stato istruttoria',StringType(),True),
           StructField('Importo approvato',StringType(),True),
           StructField('Ruolo richiedente',StringType(),True),
           StructField('Data Determinazione Ammissibilità',TimestampType(),True),
           StructField('Data Determinazione Concessione',TimestampType(),True),
           StructField('Importo concesso',StringType(),True)
       ]

finalStructD=StructType(fields=strutturaD)
df_domande = spark.read.csv("SIL Sardegna_Elenco Domande_0.4.csv", sep=";", header=True,
                     schema=finalStructD)

In [32]:
df_domande.printSchema()

root
 |-- Codice avviso: string (nullable = true)
 |-- Codice domanda: string (nullable = true)
 |-- Tipo destinatario: string (nullable = true)
 |-- Macrotipologia destinatario: string (nullable = true)
 |-- Sesso: string (nullable = true)
 |-- Forma giuridica: string (nullable = true)
 |-- Macro Settore ATECO: string (nullable = true)
 |-- Codice Settore ATECO: string (nullable = true)
 |-- Settore ATECO: string (nullable = true)
 |-- Comune residenza / Comune sede legale: string (nullable = true)
 |-- Provincia residenza / Provincia sede legale: string (nullable = true)
 |-- Comune domicilio / Comune sede operativa: string (nullable = true)
 |-- Provincia domicilio / Provincia sede operativa: string (nullable = true)
 |-- Data Invio: timestamp (nullable = true)
 |-- Importo richiesto: string (nullable = true)
 |-- Data fine istruttoria: date (nullable = true)
 |-- ID Istruttore: string (nullable = true)
 |-- Stato istruttoria: string (nullable = true)
 |-- Importo approvato: string 

In [33]:
print("Dataframe shape:\nRows: ",df_domande.count(), " Cols: ",len(df_domande.columns))

Dataframe shape:
Rows:  59025  Cols:  23


###Data preparation
Il dataset "Domande" contiene le informazioni relative a ogni domanda presentata attraverso il sistema SIL Sardegna, informazioi sul soggetto partecipante  e ulteriori informazioni circa la gestione del procedimento da parte degli uffici competenti, quali:
1. informazioni Anagrafica partecipante (Cittadino/Impresa, ubicazione, settore ATECO, ...)
2. informazioni relative all'istruttoria (Data, operatore, risultato, importo finanziabile)
2. informazioni relative alla concessione (Data e importo della concessione)

Occorre verificare la formattazione degli importi e delle date.

####Formattazione importi

In [34]:
df_domande = df_domande\
    .withColumn('Importo richiesto', regexp_replace('Importo richiesto', ',', '.'))\
    .withColumn('Importo approvato', regexp_replace('Importo approvato', ',', '.'))\
    .withColumn('Importo concesso', regexp_replace('Importo concesso', ',', '.'))

df_domande = df_domande\
    .withColumn('Importo richiesto', df_domande['Importo richiesto'].cast("float"))\
    .withColumn('Importo approvato', df_domande['Importo approvato'].cast("float"))\
    .withColumn('Importo concesso', df_domande['Importo concesso'].cast("float"))\
    .na.fill(value=0)

###Prima esplorazione dei dati

In [35]:
df_domande.show()

+--------------------+--------------+-----------------+---------------------------+---------------+--------------------+--------------------+--------------------+--------------------+-------------------------------------+-------------------------------------------+----------------------------------------+----------------------------------------------+-------------------+-----------------+---------------------+---------------+-----------------+-----------------+--------------------+---------------------------------+-------------------------------+----------------+
|       Codice avviso|Codice domanda|Tipo destinatario|Macrotipologia destinatario|          Sesso|     Forma giuridica| Macro Settore ATECO|Codice Settore ATECO|       Settore ATECO|Comune residenza / Comune sede legale|Provincia residenza / Provincia sede legale|Comune domicilio / Comune sede operativa|Provincia domicilio / Provincia sede operativa|         Data Invio|Importo richiesto|Data fine istruttoria|  ID Istruttore|S

##Join dei Datasets
I due Dataset sono "uniti" sulla base del "Codice Avviso" in modo da poter gestire l'analisi agevolmente.

In [36]:
#Spark join sulla base della feature "Codice Avviso"
df_domande_j = df_domande.join(df_avvisi, ['Codice Avviso'], how="left")


In [37]:
print("Dataframe shape:\nRows: ",df_domande_j.count(), " Cols: ",len(df_domande_j.columns))

Dataframe shape:
Rows:  59025  Cols:  43


In [38]:
df_domande_j.show()

+--------------------+--------------+-----------------+---------------------------+---------------+--------------------+--------------------+--------------------+--------------------+-------------------------------------+-------------------------------------------+----------------------------------------+----------------------------------------------+-------------------+-----------------+---------------------+---------------+-----------------+-----------------+--------------------+---------------------------------+-------------------------------+----------------+--------------------+----------------+-----------------------+--------------------+-----------------------------+----------------------------------------+-------------------------------------------------+-------------+-------------+----------+---------------------+---------------+---------------------+---------------------+----------------+-------------------+--------------+---------------+-------------------------+------------

### Selezione degli interventi COVID

In [39]:
df_domande_j =  df_domande_j.filter(df_domande_j['Intervento COVID'] == 1)
df_domande_j.count()

55666

## Analisi della presentazione delle domande (Partecipazione)
Sono analizzate le domande sulla base del perioro di presentazione e sulla ripartizione dei partecipanti, in base al numero e alle somme destinate alle diverse tipologie di target.

###Panoramica interventi COVID
Sono esplorati e visualizzati gli "interventi COVID" distinti per tipologia, budget assegnato e budget prenotato:
1. Ammortizzatori sociali
2. Domanda di Aiuto Telematica (DAT)
3. Domanda di Candidatura Telematica (DCT)
4. Domanda di finanziamento
5. Domanda di Incentivo Telematica (DIT)

In [40]:
# Solo avvisi classificati "Intervento COVID"
Avvisi_covid = df_avvisi.filter(df_avvisi['Intervento COVID'] == 1).na.fill(value=0).withColumnRenamed("N. Domande","Domande")
Avvisi_covid_pd = Avvisi_covid.toPandas()

In [41]:
Avvisi_tipologia = Avvisi_covid.filter(col("Intervento COVID")==1).groupBy("Tipologia Avviso").agg({'Dotazione Finanziaria':'sum', 'Budget prenotato':'sum'})
Avvisi_tipologia_pd = Avvisi_tipologia.toPandas()

In [42]:
tipologie = Avvisi_tipologia_pd["Tipologia Avviso"].unique()
# Create traces
fig = make_subplots(
    rows=2, cols=1,
    vertical_spacing=0.05,
    #row_heights=[100,200],
    specs=[[{"type": "table"}],
           [{'type':'bar'}]
           ]
)
fig.add_trace(go.Table(
    header=dict(
        values=["Codice Avviso", "Descrizione estesa", "Tipologia","Data di Apertura",
                    "Data di chiusura", "Dotazione<br>Finanziaria"],
            font=dict(size=10),
            align="left"),
    cells=dict(
        values=[Avvisi_covid_pd[k].tolist() for k in Avvisi_covid_pd.iloc[:,[0,1,2,8,9,11]].columns],
        align = "left"
    ),
    columnwidth= [14,30,8,7,7,7]
  ),1,1)

fig.add_trace(go.Bar(
    x= tipologie,
    y= Avvisi_tipologia_pd["sum(Dotazione Finanziaria)"],
    name= "Dotazione iniziale",
    hovertext = "Dotazione",
    marker_color='darkgrey',
    hovertemplate = '<br><b><i>%{x}</i></b>'+
    '<br><i>Dotazione Finanziaria</i>:<br><b>%{y:.2f}</b>'+
    '<extra></extra>')
,2,1)
fig.add_trace(go.Bar(
    x= tipologie,
    y= Avvisi_tipologia_pd["sum(Budget prenotato)"],
    name="Budget prenotato",
    hovertext = "Budget prenotato",
    marker_color='indianred',
    hovertemplate = '<br><b><i>%{x}</i></b>'+
    '<br><i>Budget Prenotato</i>:<br><b>%{y:.2f}</b>'+
    '<extra></extra>')
,2,1)

fig.update_yaxes(type="log")
#fig.update_xaxes(tickangle = 90)
fig.update_layout(
    title_text="Panoramica interventi COVID per tipologia")

fig.update_layout(barmode='group')

fig.show()

###Andamento nel tempo
E' analizzata la numerosità delle domande presentate giornalmente per tutto il periodo di osservazione, nel grafico sono stati riportati anche i "momenti" di apertura dei diversi Avvisi pubblici e una fascia in semi trasparenza per la durata dell'apertura dell'avviso (finestra di apertura). <br>
E' interessante notare i picchi in relazione all'apertura di alcuni avvisi che sono stati gestiti in modalità "a sportello" "Click Day".


In [43]:
Somme_richieste_daily = df_domande_j.na.fill(value=0).groupBy("Data Invio").agg({'Importo Richiesto':'sum', 'Codice Domanda':'count'})
Somme_richieste_daily_pd = Somme_richieste_daily.sort("Data Invio").toPandas()


In [44]:
Somme_richieste_daily_pd['Data formatted'] = Somme_richieste_daily_pd['Data Invio'].dt.strftime('%m/%d/%Y')
#Somme_richieste_daily_pd.head()

In [45]:
Avvisi_covid_pd.sort_values("Data Apertura", ascending = True,
                 inplace = True, na_position ='last')

In [46]:
avvisi = Avvisi_covid_pd["Codice Avviso"].unique()
fig = px.line(Somme_richieste_daily_pd, x="Data Invio", y='count(Codice Domanda)',
              title='Andamento presentazione delle domande', 
              #line_group= df1.index.get_level_values(0), 
              labels={ # replaces default labels by column name
                     "count(Codice Domanda)": "N. Domande", "sum(Importo Richiesto)": "Somma Importi (€)"},
              #log_y = True,
              hover_name= "Data formatted",
              hover_data=["count(Codice Domanda)","sum(Importo Richiesto)"]

            )
fig.add_trace(go.Scatter(x=Avvisi_covid_pd["Data Apertura"], y=Avvisi_covid_pd["Intervento COVID"],
                mode='markers',
                name='Apertura avviso',
                hovertext = "Femmine",
                marker_color='indianred',
                hovertemplate = '<br><b><i>%{x}</i></b>'+
                '<br><i>Avviso</i>:<br> <b>'+ Avvisi_covid_pd["Codice Avviso"] +'</b>'+
                '<extra></extra>'
                )
)
for index, avviso in enumerate(avvisi):
    fig.add_vrect(x0=Avvisi_covid_pd.loc[Avvisi_covid_pd["Codice Avviso"] == avviso, "Data Apertura"].item().strftime('%Y-%m-%d'), 
                  x1=Avvisi_covid_pd.loc[Avvisi_covid_pd["Codice Avviso"] == avviso, "Data Chiusura"].item().strftime('%Y-%m-%d'), 
                  #annotation_text=Avvisi_covid_pd.loc[Avvisi_covid_pd["Codice Avviso"] == avviso, "Codice Avviso"].item(), annotation_position="top left", 
                  fillcolor="green", opacity=0.10, line_width=0, 
                  #annotation_textangle=-45
                  )

fig.show()


###Erogazione dei fondi
E' esplorato l'andamento generale di tre elementi relativi ai fondi tracciati sul SIL Sardegna:
1. Budget Prenotato (somma degli importi richiesti attraverso presentazione di domanda telematica)
2. Budget Approvato (somma degli importi approvati attraverso esito istruttorio)
3. Budget Concesso (somma degli importi che sono stati inseriti in determina di concessione verso i beneficiari)


In [47]:
Domande_richiesto = df_domande_j.na.fill(value=0)\
            .filter(col("Istruttoria extra-sistema")=="NO")\
            .groupBy(month("Data Invio"), year("Data Invio"))\
            .agg({'Importo Richiesto':'sum'})
Domande_approvato = df_domande_j.na.fill(value=0).groupBy(month("Data fine istruttoria"), year("Data fine istruttoria"))\
            .agg({'Importo approvato':'sum'})
Domande_concesso = df_domande_j.na.fill(value=0).groupBy(month("Data Determinazione Ammissibilità"), year("Data Determinazione Ammissibilità"))\
            .agg({'Importo concesso':'sum'})
Stat_domande = Domande_richiesto.join(Domande_approvato, (Domande_richiesto['month(Data Invio)']==Domande_approvato['month(Data fine istruttoria)'])\
                                      & (Domande_richiesto['year(Data Invio)']==Domande_approvato['year(Data fine istruttoria)']), how='left')\
                                      .join(Domande_concesso, (Domande_richiesto['month(Data Invio)']==Domande_concesso['month(Data Determinazione Ammissibilità)'])\
                                            & (Domande_richiesto['year(Data Invio)']==Domande_concesso['year(Data Determinazione Ammissibilità)']), how = 'left')
#Aggiungo la colonna "Mese-Anno"
Stat_domande = Stat_domande.na.fill(0).withColumn("Mese-Anno", concat(col("month(Data Invio)"), lit("/"),col("year(Data Invio)"))).sort(["year(Data Invio)","month(Data Invio)"])


In [48]:
#Calcolo le somme cumulative usando le "Finestre" Spark

from pyspark.sql import Window

window = Window.orderBy(["year(Data Invio)","month(Data Invio)"])
  

df_domande_cumulative = Stat_domande\
                .withColumn('Avanzamento Prenotato', sum('sum(Importo Richiesto)').over(window))\
                .withColumn('Avanzamento Approvato', sum('sum(Importo approvato)').over(window))\
                .withColumn('Avanzamento Concesso', sum('sum(Importo concesso)').over(window))
#df_domande_cumulative.show()

In [49]:
Stat_domande_pd = df_domande_cumulative.toPandas()

In [50]:
#Stat_domande_pd.head()

In [64]:
x = Stat_domande_pd["Mese-Anno"]
y1 = Stat_domande_pd["Avanzamento Prenotato"]
y2 = Stat_domande_pd["Avanzamento Approvato"]
y3 = Stat_domande_pd["Avanzamento Concesso"]

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y1,
                    mode='lines',
                    fill="tozeroy",
                    name='Budget prenotato',
                    hovertemplate = '<br><b><i>%{x}</i></b>'+
                    '<br><i>Budget prenotao</i>:<br>€ <b>%{y:.2f}</b>'+
                    '<extra></extra>', # Rimuove il trace durante l'hover
                    ))
fig.add_trace(go.Scatter(x=x, y=y2,
                    mode='lines',
                    fill="tozeroy",
                    name='Budget approvato',
                    hovertemplate = '<br><b><i>%{x}</i></b>'+
                    '<br><i>Budget approvato</i>:<br>€ <b>%{y:.2f}</b>'+
                    '<extra></extra>', # Rimuove il trace durante l'hover
                    ))
fig.add_trace(go.Scatter(x=x, y=y3,
                    mode='lines',
                    fill="tozeroy",
                    name='Budget concesso',
                    hovertemplate = '<br><b><i>%{x}</i></b>'+
                    '<br><i>Budget concesso</i>:<br>€ <b>%{y:.2f}</b>'+
                    '<extra></extra>', # Rimuove il trace durante l'hover
                    ))

fig.update_yaxes(type="linear", title_text="Somme (€)")
#fig.update_yaxes(title_text="Somme (€)")
fig.update_layout(title="Confronto avanzamento pratiche", hovermode="x")

fig.show()

###Confronto stato istruttoria
L'esplorazione è condotta sui soli procedimenti la cui fase istruttoria è gestita attraverso gli strumenti del SIL Sardegna (Istruttoria extra-sistema == NO), con lo scopo di verificare, oltre che lo stato di avanzamento, eventuali anomalie nelle procedure. <br> In particolare si noterà che l'avviso di cui all'Art. 9  della LR 22 presenta una percentuale di istruttorie esitate negativamente pari a oltre il 48%, questa situazione meriterebbe un approfondimento e una riflessione sulle cause di tale fenomeno.


In [52]:
Domande_per_istruttoria = df_domande_j.na.fill(value=0)\
        .withColumnRenamed("N. domande","Domande")\
        .filter(col("Istruttoria extra-sistema")=="NO")\
        .groupBy("Codice Avviso","Domande")\
        .pivot("Stato Istruttoria")\
        .agg({'Codice Domanda': 'count'})

Domande_per_istruttoria.show(30)

+--------------------+-------+--------+--------+---------------+--------+
|       Codice Avviso|Domande|In corso|Negativa|Non Disponibile|Positiva|
+--------------------+-------+--------+--------+---------------+--------+
|(R)ESISTO - Linea...|   9215|    8064|      72|           null|    1079|
|  Fondo di rotazione|    242|    null|    null|            242|    null|
|            LR22ART9|  11006|     518|    5355|           null|    5135|
|SSE Lavoro Sardeg...|    103|    null|    null|            103|    null|
|CIG Deroga 2020 c...|  16979|    null|    1304|           null|   15675|
|SSE Lavoro Sardeg...|    992|    null|    null|            992|    null|
|(R)ESISTO - Linea...|   7425|    6426|     106|           null|     893|
|(R)ESISTO - Linea...|     15|      15|    null|           null|    null|
+--------------------+-------+--------+--------+---------------+--------+



In [53]:
Domande_per_istruttoria_pd = Domande_per_istruttoria.na.fill(0).orderBy("Codice Avviso").toPandas()

In [54]:
avvisi = Domande_per_istruttoria_pd["Codice Avviso"].unique()
Domande_per_istruttoria_pd["Non disp perc"] = Domande_per_istruttoria_pd["Non Disponibile"]/Domande_per_istruttoria_pd["Domande"] * 100
Domande_per_istruttoria_pd["In cors perc"] = Domande_per_istruttoria_pd["In corso"]/Domande_per_istruttoria_pd["Domande"] * 100
Domande_per_istruttoria_pd["Pos perc"] = Domande_per_istruttoria_pd["Positiva"]/Domande_per_istruttoria_pd["Domande"] * 100
Domande_per_istruttoria_pd["Neg perc"] = Domande_per_istruttoria_pd["Negativa"]/Domande_per_istruttoria_pd["Domande"] * 100
Domande_per_istruttoria_pd = Domande_per_istruttoria_pd.round({'Non disp perc': 2, 'In cors perc': 2, 'Pos perc': 2, 'Neg perc': 2})
fig = go.Figure()
fig.add_trace(go.Bar(
    y=avvisi,
    x=Domande_per_istruttoria_pd["Non disp perc"],
    name='Istanza da lavorare',
    orientation='h',
    hovertemplate = '<br><b><i>%{y}</i></b>'+
    '<br><i>% da lavorare</i>: <b>%{x:.2f}</b>'+
    '<extra></extra>', # Rimuove il trace durante l'hover
    text= Domande_per_istruttoria_pd["Non disp perc"],
    textposition='inside'
))
fig.add_trace(go.Bar(
    y=avvisi,
    x=Domande_per_istruttoria_pd["In cors perc"],
    name='Istruttoria in corso',
    orientation='h',
    hovertemplate = '<br><b><i>%{y}</i></b>'+
    '<br><i>% in corso</i>: <b>%{x:.2f}</b>'+
    '<extra></extra>', # Rimuove il trace durante l'hover
    text= Domande_per_istruttoria_pd["In cors perc"],
    textposition='inside'
))

fig.add_trace(go.Bar(
    y=avvisi,
    x=Domande_per_istruttoria_pd["Pos perc"],
    name='Istruttoria positiva',
    orientation='h',
    hovertemplate = '<br><b><i>%{y}</i></b>'+
    '<br><i>% positive</i>: <b>%{x:.2f}</b>'+
    '<extra></extra>', # Rimuove il trace durante l'hover
    text= Domande_per_istruttoria_pd["Pos perc"],
    textposition='inside'
))


fig.add_trace(go.Bar(
    y=avvisi,
    x=Domande_per_istruttoria_pd["Neg perc"],
    name='Istruttoria negativa',
    orientation='h',
    hovertemplate = '<br><b><i>%{y}</i></b>'+
    '<br><i>% negative</i>: <b>%{x:.2f}</b>'+
    '<extra></extra>', # Rimuove il trace durante l'hover
    text= Domande_per_istruttoria_pd["Neg perc"],
    textposition='inside'
))

fig.update_xaxes(title_text="%", type="linear")
fig.update_yaxes(title_text="Avviso")
fig.update_layout(title="Confronto avanzamento dell'attività istruttoria")
fig.update_layout(barmode='stack', 
                  #hovermode="x", 
                  hovermode = "closest",
                  hoverlabel= { 'bgcolor': "#FFF" }
)
fig.show()

###Tipologia di partecipanti

Sono analizzate alcune caratteristiche principali dei soggetti che hanno presentato istanza di partecipazione. L'analisi è distinta per due macro tipologie: Cittadini e Imprese/Enti.



In [55]:
Stat_partecipanti = df_domande_j.na.fill(value=0).groupBy(["Macrotipologia destinatario"]).agg({'Importo Richiesto':'sum', 'Codice Domanda':'count'})
Stat_partecipanti_pd = Stat_partecipanti.toPandas()

In [56]:
Stat_partecipanti_pd.head(30)

,Macrotipologia destinatario,sum(Importo Richiesto),count(Codice Domanda)
0,Imprese / Enti,3.949117e+08,44226
1,Cittadini,1.384740e+07,11440


In [65]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2, subplot_titles=("N. di Domande presentate", "Budget richiesto (€)"), specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=Stat_partecipanti_pd['Macrotipologia destinatario'], values=Stat_partecipanti_pd['count(Codice Domanda)']), 1, 1)

fig.add_trace(go.Pie(labels=Stat_partecipanti_pd['Macrotipologia destinatario'], values=Stat_partecipanti_pd['sum(Importo Richiesto)']), 1, 2)

fig.update_traces(hole=.4, hoverinfo="label+value")
fig.update_layout(
    title_text="Confronto partecipanti")
fig.show()

###Ripartizione Cittadini

In [58]:
Stat_cittadini = df_domande_j.orderBy("Codice Avviso").na.fill(value=0).filter(df_domande_j['Tipo destinatario'] == "Cittadini").groupBy(["Codice Avviso","Sesso"]).agg({'Importo Richiesto':'sum', 'Codice Domanda':'count'})
Stat_cittadini_pd = Stat_cittadini.toPandas()

In [59]:
Stat_cittadini_pd.head(20)

,Codice Avviso,Sesso,sum(Importo Richiesto),count(Codice Domanda)
0,LR22ART8,M,0.000000e+00,264
1,LR22ART8,F,0.000000e+00,24
2,LR22ART9,F,7.505415e+06,5976
3,LR22ART9,M,6.275201e+06,5032
4,LR22ART9D,F,4.805882e+04,111
5,LR22ART9D,M,1.872745e+04,33


In [66]:
avvisi = Stat_cittadini_pd["Codice Avviso"].unique()

fig = make_subplots(rows=1, cols=2, subplot_titles=("N. di domande presentate", "Budget richiesto (€)"), specs=[[{'type':'bar'}, {'type':'bar'}]])

fig.add_trace(go.Bar(
    x= avvisi,
    y= Stat_cittadini_pd.loc[(Stat_cittadini_pd["Sesso"] =="M"),  "count(Codice Domanda)"],
    name= "Maschi",
    hovertext = "Maschi",
    marker_color='darkgrey',
    hovertemplate = '<br><b><i>%{x}</i></b>'+
    '<br><i>N. Maschi</i>: <b>%{y}</b>'+
    '<extra></extra>', # Rimuove il trace durante l'hover
    legendgroup ="maschi")
,1,1)
fig.add_trace(go.Bar(
    x= avvisi,
    y=Stat_cittadini_pd.loc[(Stat_cittadini_pd["Sesso"] =="F"),  "count(Codice Domanda)"],
    name="Femmine",
    hovertext = "Femmine",
    marker_color='indianred',
    hovertemplate = '<br><b><i>%{x}</i></b>'+
    '<br><i>N. Femmine</i>: <b>%{y}</b>'+
    '<extra></extra>', # Rimuove il trace durante l'hover
    legendgroup ="Femmine")
,1,1)
fig.add_trace(go.Bar(
    x= avvisi,
    y=Stat_cittadini_pd.loc[(Stat_cittadini_pd["Sesso"] =="M"),  "sum(Importo Richiesto)"],
    name= "Maschi",
    hovertext = "Maschi",
    marker_color='darkgrey',
    hovertemplate = '<br><b><i>%{x}</i></b>'+
    '<br><i>Importi Maschi</i>:<br>€ <b>%{y:.2f}</b>'+
    '<extra></extra>', # Rimuove il trace durante l'hover
    legendgroup ="maschi",
    showlegend=False)
,1,2)
fig.add_trace(go.Bar(
    x= avvisi,
    y=Stat_cittadini_pd.loc[(Stat_cittadini_pd["Sesso"] =="F"),  "sum(Importo Richiesto)"],
    name="Femmine",
    hovertext = "Femmine",
    marker_color='indianred',
    hovertemplate = '<br><b><i>%{x}</i></b>'+
    '<br><i>Importi Femmine</i>:<br>€ <b>%{y:.2f}</b>'+
    '<extra></extra>', # Rimuove il trace durante l'hover
    legendgroup ="Femmine",
    showlegend=False)
,1,2)
fig.update_yaxes(type="log")
fig.update_xaxes(tickangle = -45)
fig.update_layout(
    title_text="Confronto cittadini per genere")

fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

###Ripartizione Enti e Imprese

In [61]:
Stat_imprese = df_domande_j.na.fill(value=0).filter(df_domande_j['Macrotipologia destinatario'] == "Imprese / Enti").groupBy(["Codice Avviso","Macro Settore Ateco"]).agg({'Importo Richiesto':'sum', 'Codice Domanda':'count'})
Stat_imprese_pd = Stat_imprese.toPandas()

In [62]:
#Stat_imprese_pd.head(10)

In [67]:
avvisi = Stat_imprese_pd["Codice Avviso"].unique()
totale_domande = Stat_imprese_pd['count(Codice Domanda)'].sum()
totale_finanziamento = currency_converter(Stat_imprese_pd['sum(Importo Richiesto)'].sum())

fig = make_subplots(rows=1, cols=2, subplot_titles=("N. di domande presentate", "Budget richiesto (€)"), specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=avvisi, values=Stat_imprese_pd.groupby(['Codice Avviso']).sum()['count(Codice Domanda)']), 1, 1)

fig.add_trace(go.Pie(labels=avvisi, values=Stat_imprese_pd.groupby(['Codice Avviso']).sum()['sum(Importo Richiesto)']), 1, 2)

fig.update_traces(hole=.4, hoverinfo="label+value")
fig.update_layout(
    title_text="Confronto partecipazione imprese"#,
    #annotations=[dict(text=int(totale_domande), x=0.225, y=0.46, font_size=20, showarrow=False,),
    #             dict(text=totale_finanziamento, x=0.775, y=0.47, font_size=14, showarrow=False)]
    )
fig.show()